# Root-finding using Python

In [1]:
import numpy as np
import scipy.stats as ss
import scipy.optimize as sopt
import datetime as dt

In [2]:
def myFunc(x):
    return( x**3 + 2*x - 10 )

In [3]:
x = sopt.brentq(myFunc, 1, 3)
print(x, myFunc(x))

1.8474190378327324 -3.552713678800501e-15


In [4]:
# What if the intervals are not good?
sopt.brentq(myFunc, 1, 1.5)

ValueError: f(a) and f(b) must have different signs

## Binding Arguments 

### 1. Use functools.partial

In [5]:
# You want to solve myFunc(x) - y = 0
# How we do define a function with arbitrary y?

def myFuncY(y,x):
    return myFunc(x) - y

In [6]:
import functools
# functools.partial 'binds' the first argument to a specific value
myFunc0 = functools.partial(myFuncY,0)

In [7]:
myFunc(2), myFunc0(2)

(2, 2)

In [8]:
myFunc5 = functools.partial(myFuncY,5)

In [9]:
# Let's solve root
x = sopt.brentq(myFunc5, 1, 3)
print(x, myFunc5(x))

2.1970910517887083 -7.105427357601002e-15


### 2. Use lambda

In [10]:
# lambda is used for a quick definition of a function
g = lambda x: x**2
g(5)

25

In [11]:
# See the difference between function and lambda
myFunc, g

(<function __main__.myFunc>, <function __main__.<lambda>>)

### Lambdas capture the variable, partial captures the value.
A cool thing about lambda

In [12]:
y = 0
myFunc0 = functools.partial(myFuncY, y)
g = lambda x: myFuncY(y,x)
myFunc0(2), g(2)

(2, 2)

In [13]:
# now let's change y
y = 5
myFunc0(2), g(2)

(2, -3)

## Solving Implied Volatility

In [14]:
def bsm_option_price(strike, spot, vol, texp, intr=0.0, divr=0.0, cp_sign=1):
    vol_std = vol * np.sqrt(texp)
    div_fac = np.exp(-texp*divr)
    disc_fac = np.exp(-texp*intr)
    forward = spot / disc_fac * div_fac
    d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
    d2 = d1 - vol_std

    price = cp_sign * disc_fac \
        * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
    return price

In [15]:
strike = 105
spot = 100
vol = 0.2
texp = 0.25

price = bsm_option_price(strike, spot, vol, texp)
print(price)

2.0640191379


In [16]:
# We bind all the arguments EXCEPT vol
func_impvol = lambda _vol: bsm_option_price(strike, spot, _vol, texp) - price

In [17]:
# Let's find the implied vol
impvol = sopt.brentq(func_impvol, 0.0001, 1)
print('{:0.2f}%'.format(impvol*100))

20.00%


In [18]:
# For different price
price = 3
impvol = sopt.brentq(func_impvol, 0.0001, 1)
print('{:0.2f}%'.format(impvol*100))

25.04%


### Class version

In [19]:
class OptionContract:
    def __init__(self, undl, opt_type, strike, dexp):
        ''' Constructor for this class. '''
        self.undl, self.strike, self.dexp = undl, strike, dexp
        self.opt_type = opt_type
        self.cp_sign = 1 if (opt_type == 'call') else -1

    def toString(self):
        return('{:s} option on {:s} struck at {:0.1f} maturing on {:s}'\
              .format(self.opt_type, self.undl, self.strike, self.dexp.strftime('%Y.%m.%d')))

    def price(self, spot, model):
        return model.price_contract(spot, self)


In [20]:
class BsmModel:
    vol, intr, divr = None, None, None

    def __init__(self, vol, intr=0.0, divr=0.0): # Constructor
        self.set_params(vol, intr, divr)

    def set_params(self, vol=None, intr=None, divr=None):
        self.vol = vol if(vol != None) else self.vol
        self.intr = intr if(intr != None) else self.intr
        self.divr = divr if(divr != None) else self.divr

    def price(self, strike, spot, texp, cp_sign=1):
        return bsm_option_price(strike, spot, self.vol, texp, intr=self.intr, divr=self.divr, cp_sign=1)

    def price_contract(self, spot, contract):
        texp = (contract.dexp - dt.date.today()).days/365.25
        return self.price(contract.strike, spot, texp, contract.cp_sign)
    
    def impvol(self, price, strike, spot, texp, cp_sign=1):
        iv_func = lambda _vol: \
            bsm_option_price(strike, spot, _vol, texp, self.intr, self.divr, cp_sign) - price
        vol = sopt.brentq(iv_func, 0.00001, 10)
        return vol
    
    def impvol_contract(self, contract, price ):
        texp = (contract.dexp - dt.date.today()).days/365.25
        return self.impvol(price, contract.strike, spot, texp, contract.cp_sign)

    def calibrate(self, contract, price):
        self.vol = self.impvol_contract(contract, price)
        print('vol set to {:0.1f}%'.format(self.vol*100))

In [21]:
bsm3 = BsmModel(vol=0.2)

price = bsm3.price(strike=102, spot=100, texp=1, cp_sign=1)
price

7.0844942478298947

In [22]:
bsm3.impvol(price, strike=102, spot=100, texp=1)

0.20000000000003704

In [23]:
tc_c105_dec = OptionContract('Tencent', 'call', 105, dexp=dt.date(2017, 12, 25))
tc_p95_dec = OptionContract('Tencent', 'put', 95, dexp=dt.date(2017, 12, 25))
print(tc_c105_dec.toString())
print(tc_p95_dec.toString())

call option on Tencent struck at 105.0 maturing on 2017.12.25
put option on Tencent struck at 95.0 maturing on 2017.12.25


In [24]:
tc_spot = 100
price1 = tc_c105_dec.price(tc_spot, model=bsm3)
price2 = tc_p95_dec.price(tc_spot, model=bsm3)
print(price1, price2)

2.05781267056 6.88223333229


In [25]:
# Let's assume price1 and price2 are given from market
price1 = 2.9
vol1 = bsm3.impvol_contract(tc_c105_dec, price1)
price2 = 8
vol2 = bsm3.impvol_contract(tc_p95_dec, price2)

print('{:0.2f}% {:0.2f}%'.format(vol1*100, vol2*100))

24.56% 53.26%


In [26]:
bsm3.calibrate(tc_c105_dec, price1)
tc_c105_dec.price(tc_spot, model=bsm3)

vol set to 24.6%


2.8999999999999986

In [27]:
1-np.pi/4

0.21460183660255172